In [3]:
# ===============================================================
# Physics-Constrained MGB for Colebrook Friction Factor (Colab)
# Single-cell script: data gen -> training -> UQ -> eval -> plots -> downloads
# ===============================================================

# 0) Setup
!pip -q install lightgbm==4.5.0

import numpy as np, pandas as pd, matplotlib.pyplot as plt, math, os, io, json, zipfile, sys, time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from lightgbm import LGBMRegressor

# Colab download helper
IN_COLAB = False
try:
    from google.colab import files
    IN_COLAB = True
except Exception:
    pass

np.random.seed(42)

# ---------------------------------------------------------------
# 1) Colebrook–White solver (Newton on x = 1/sqrt(f), init Serghides)
# ---------------------------------------------------------------
def serghides_approx_f(Re, k_rel):
    Re = np.asarray(Re, dtype=float)
    k_rel = np.asarray(k_rel, dtype=float)
    A = -2.0*np.log10(k_rel/3.7 + 12.0/Re)
    B = -2.0*np.log10(k_rel/3.7 + 2.51*A/Re)
    C = -2.0*np.log10(k_rel/3.7 + 2.51*B/Re)
    return 1.0/(C*C)

def colebrook_white(Re, k_rel, max_iter=30, tol=1e-12):
    Re = np.asarray(Re, dtype=float)
    k_rel = np.asarray(k_rel, dtype=float)
    f0 = serghides_approx_f(Re, k_rel)
    x = 1.0/np.sqrt(np.clip(f0, 1e-16, None))  # x = 1/sqrt(f)
    ln10 = np.log(10.0)
    for _ in range(max_iter):
        denom = k_rel/3.7 + 2.51/(Re*x)
        g = x + 2.0*np.log10(np.clip(denom, 1e-300, None))
        dg = 1.0 + 2.0*(1.0/ln10) * ( (-2.51/(Re*np.clip(x**2,1e-300,None))) / np.clip(denom,1e-300,None) )
        step = g/np.clip(dg, 1e-16, None)
        x_new = x - step
        if np.max(np.abs(step)) < tol:
            x = x_new
            break
        x = x_new
    f = 1.0/np.clip(x**2, 1e-30, None)
    return f

# ---------------------------------------------------------------
# 2) Synthetic data (train grid dense, eval grid = paper's 24×7)
# ---------------------------------------------------------------
RE_MIN, RE_MAX = 4e3, 2.5e7
N_RE_TRAIN = 240
N_K_TRAIN  = 61
re_train = np.logspace(np.log10(RE_MIN), np.log10(RE_MAX), N_RE_TRAIN)
k_train  = np.linspace(0.0, 0.06, N_K_TRAIN)

Re_mesh, k_mesh = np.meshgrid(re_train, k_train, indexing='xy')
Re_flat = Re_mesh.ravel()
k_flat  = k_mesh.ravel()
f_true  = colebrook_white(Re_flat, k_flat)

df = pd.DataFrame({"Re": Re_flat, "k_rel": k_flat, "f_colebrook": f_true})
df["log10Re"] = np.log10(df["Re"].values)

# Train/Calib/Test split (70/15/15) stratified by roughness bins
df["k_bin"] = pd.cut(df["k_rel"], bins=10, labels=False, include_lowest=True)
train_df, rest_df = train_test_split(df, test_size=0.30, random_state=42, stratify=df["k_bin"])
calib_df, test_df = train_test_split(rest_df, test_size=0.50, random_state=42, stratify=rest_df["k_bin"])

X_train = train_df[["log10Re","k_rel"]].values
y_train = train_df["f_colebrook"].values
X_calib = calib_df[["log10Re","k_rel"]].values
y_calib = calib_df["f_colebrook"].values
X_test  = test_df[["log10Re","k_rel"]].values
y_test  = test_df["f_colebrook"].values

# ---------------------------------------------------------------
# 3) Monotonic Gradient Boosting (point model has constraints)
#    Constraints: log10Re ↓ (-1), k_rel ↑ (+1)
# ---------------------------------------------------------------
point = LGBMRegressor(
    n_estimators=1200, learning_rate=0.05, num_leaves=64, min_child_samples=40,
    subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0,
    objective="l2", monotone_constraints=[-1, +1], random_state=42
)
point.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric="l2")

# Quantile models for UQ (NO monotone_constraints here!)
alpha = 0.05
ql = LGBMRegressor(
    n_estimators=800, learning_rate=0.05, num_leaves=64, min_child_samples=40,
    subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0,
    objective="quantile", alpha=alpha, random_state=42
)
qu = LGBMRegressor(
    n_estimators=800, learning_rate=0.05, num_leaves=64, min_child_samples=40,
    subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0,
    objective="quantile", alpha=1.0-alpha, random_state=42
)
ql.fit(X_train, y_train)
qu.fit(X_train, y_train)

# ---------------------------------------------------------------
# 4) Split Conformal Calibration (two-sided CQR)
# ---------------------------------------------------------------
lower_cal = ql.predict(X_calib)
upper_cal = qu.predict(X_calib)
s = np.maximum(lower_cal - y_calib, y_calib - upper_cal)
q_hat = np.quantile(s, 1.0 - alpha)

def calibrated_interval(X):
    lo = ql.predict(X) - q_hat
    up = qu.predict(X) + q_hat
    return np.minimum(lo, up), np.maximum(lo, up)

# ---------------------------------------------------------------
# 5) Evaluation: metrics, monotonicity checks, coverage, widths
# ---------------------------------------------------------------
y_pred = point.predict(X_test)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
mae  = mean_absolute_error(y_test, y_pred)
mape = float(np.mean(np.abs((y_test - y_pred) / np.clip(y_test, 1e-12, None))) * 100.0)
maxerr = float(np.max(np.abs(y_test - y_pred)))

lo_test, up_test = calibrated_interval(X_test)
covered = ((y_test >= lo_test) & (y_test <= up_test)).astype(int)
coverage = covered.mean() * 100.0
widths = (up_test - lo_test)
width_mean = float(np.mean(widths))
width_median = float(np.median(widths))

def count_monotonic_violations(model, n_Re=60, n_k=31):
    re_vals = np.logspace(np.log10(RE_MIN), np.log10(RE_MAX), n_Re)
    k_vals  = np.linspace(0.0, 0.06, n_k)
    # Along Re (should decrease)
    v_re = 0; n_re_steps = 0
    for k in k_vals:
        X_line = np.column_stack([np.log10(re_vals), np.full_like(re_vals, k)])
        pred = model.predict(X_line)
        v_re += int(np.any(np.diff(pred) > 1e-12))
        n_re_steps += (len(pred)-1)
    # Along k (should increase)
    v_k = 0; n_k_steps = 0
    for r in re_vals:
        X_line = np.column_stack([np.full_like(k_vals, np.log10(r)), k_vals])
        pred = model.predict(X_line)
        v_k += int(np.any(np.diff(pred) < -1e-12))
        n_k_steps += (len(pred)-1)
    return v_re, n_re_steps, v_k, n_k_steps

v_re, nrs, v_k, nks = count_monotonic_violations(point)
mono_violation_summary = {
    "any_violation_along_Re_lines": int(v_re > 0),
    "any_violation_along_k_lines": int(v_k > 0),
    "checked_Re_steps": int(nrs),
    "checked_k_steps": int(nks)
}

# ---------------------------------------------------------------
# 6) Paper's 24×7 evaluation grid (for reporting & figures)
# ---------------------------------------------------------------
re_eval = np.logspace(np.log10(RE_MIN), np.log10(RE_MAX), 24)
k_eval  = np.array([0.00,0.01,0.02,0.03,0.04,0.05,0.06])
Re_e, k_e = np.meshgrid(re_eval, k_eval, indexing='xy')
X_eval = np.column_stack([np.log10(Re_e.ravel()), k_e.ravel()])
y_eval = colebrook_white(Re_e.ravel(), k_e.ravel())

y_hat = point.predict(X_eval)
lo_e, up_e = calibrated_interval(X_eval)
covered_e = ((y_eval >= lo_e) & (y_eval <= up_e)).astype(int)

eval_df = pd.DataFrame({
    "Re": Re_e.ravel(), "k_rel": k_e.ravel(),
    "f_colebrook": y_eval,
    "f_pred": y_hat,
    "PI_lower": lo_e, "PI_upper": up_e,
    "covered": covered_e
})
eval_df["abs_err"]  = np.abs(eval_df["f_pred"] - eval_df["f_colebrook"])
eval_df["pct_err"]  = 100.0 * eval_df["abs_err"] / np.clip(eval_df["f_colebrook"], 1e-12, None)

# ---------------------------------------------------------------
# 7) Plots (PNG) + exports
# ---------------------------------------------------------------
os.makedirs("mgb_outputs", exist_ok=True)

# Parity (test)
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, s=8, alpha=0.5)
mmin, mmax = float(min(y_test.min(), y_pred.min())), float(max(y_test.max(), y_pred.max()))
plt.plot([mmin, mmax], [mmin, mmax], lw=1)
plt.xlabel("Colebrook f"); plt.ylabel("MGB predicted f"); plt.title("Parity (Test)")
plt.tight_layout(); plt.savefig("mgb_outputs/parity_test.png", dpi=200); plt.close()

# Residual vs Re (test)
plt.figure(figsize=(7,4))
plt.scatter(10**X_test[:,0], (y_pred - y_test), s=6, alpha=0.5)
plt.xscale("log"); plt.xlabel("Re"); plt.ylabel("Residual (pred - true)"); plt.title("Residual vs Re (Test)")
plt.tight_layout(); plt.savefig("mgb_outputs/residual_vs_re_test.png", dpi=200); plt.close()

# Coverage diagnostic (test)
plt.figure(figsize=(6,4))
idx = np.argsort(y_pred)
plt.plot(np.arange(len(idx)), y_test[idx]-lo_test[idx], lw=1, label="y - L")
plt.plot(np.arange(len(idx)), up_test[idx]-y_test[idx], lw=1, label="U - y")
plt.legend(); plt.title(f"Two-sided Coverage ≈ {coverage:.2f}% (nominal 95%)")
plt.ylabel("Distance to bounds"); plt.xlabel("samples (sorted by prediction)")
plt.tight_layout(); plt.savefig("mgb_outputs/coverage_diag.png", dpi=200); plt.close()

# Interval width vs Re (24×7 grid)
plt.figure(figsize=(7,4))
plt.scatter(eval_df["Re"], (eval_df["PI_upper"]-eval_df["PI_lower"]), s=10, alpha=0.6)
plt.xscale("log"); plt.xlabel("Re"); plt.ylabel("Prediction Interval Width")
plt.title("Interval Width vs Re (24×7)")
plt.tight_layout(); plt.savefig("mgb_outputs/width_vs_re_eval.png", dpi=200); plt.close()

# CSV + JSON + ZIP
eval_csv_path = "mgb_outputs/mgb_eval_24x7.csv"
eval_df.to_csv(eval_csv_path, index=False)

summary = {
    "RMSE": rmse,
    "MAE": mae,
    "MAPE_percent": mape,
    "MaxAbsError": maxerr,
    "Coverage_percent_test": coverage,
    "IntervalWidth_mean": width_mean,
    "IntervalWidth_median": width_median,
    "Monotonicity": mono_violation_summary
}
with open("mgb_outputs/summary.json","w") as f:
    json.dump(summary, f, indent=2)

zip_path = "mgb_outputs/mgb_results.zip"
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
    for fname in ["parity_test.png","residual_vs_re_test.png","coverage_diag.png","width_vs_re_eval.png",
                  "mgb_eval_24x7.csv","summary.json"]:
        zf.write(os.path.join("mgb_outputs", fname), arcname=fname)

print("\n==== SUMMARY (copy into Results 3.7–3.9) ====")
print(json.dumps(summary, indent=2))

if IN_COLAB:
    print("\nPreparing downloads...")
    files.download(zip_path)
    files.download(eval_csv_path)
    files.download("mgb_outputs/summary.json")
else:
    print(f"\nArtifacts saved under: {os.path.abspath('mgb_outputs')}")


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000544 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 302
[LightGBM] [Info] Number of data points in the train set: 10248, number of used features: 2
[LightGBM] [Info] Start training from score 0.054701
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000433 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 302
[LightGBM] [Info] Number of data points in the train set: 10248, number of used features: 2
[LightGBM] [Info] Start training from score 0.024113
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>